## Import các thư viện cần thiết

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

---

## Thu thập dữ liệu

Trong bài này, ta dùng bộ dữ liệu được lấy từ trang [Wolrdometer](https://www.worldometers.info/coronavirus/)

Ta được phép sử dụng bộ dữ liệu này cho mục đích học tập.

---

## Khám phá dữ liệu

### Đọc dữ liệu từ file csv vào dataframe

In [3]:
data = pd.read_csv('data/covid_data.csv')
data.head()

,Date,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious Critical,Tot Cases/1M pop,Deaths/1M pop,Total Tests,Tests/1M pop,Population
0,2022-04-18,USA,82316348,NaN,1015451.0,NaN,80202314.0,NaN,1098583.0,1459.0,246106.0,3036.0,995859564.0,2977384.0,3.344747e+08
1,2022-04-18,India,43044280,NaN,521965.0,NaN,42510773.0,NaN,11542.0,698.0,30652.0,372.0,832104846.0,592552.0,1.404273e+09
2,2022-04-18,Brazil,30252618,NaN,662011.0,NaN,29227051.0,NaN,363556.0,8318.0,140538.0,3075.0,63776166.0,296272.0,2.152622e+08
3,2022-04-18,France,27771024,NaN,144157.0,NaN,24914871.0,NaN,2711996.0,1541.0,423779.0,2200.0,260504402.0,3975232.0,6.553188e+07
4,2022-04-18,Germany,23438293,NaN,133429.0,NaN,20085800.0,205500.0,3219064.0,1980.0,278156.0,1583.0,122332384.0,1451790.0,8.426317e+07


### Dữ liệu gồm bao nhiều dòng, bao nhiêu cột?

In [5]:
num_rows = data.shape[0]
num_cols = data.shape[1]
print('Số dòng: ', num_rows)
print('Số cột: ', num_cols)

Số dòng:  228
Số cột:  15


### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Mỗi dòng tương ứng với thông tin số ca nhiễm của một nước trong một ngày. 

Có vẻ như không xảy ra các dòng có ý nghĩa khác nhau

### Dữ liệu có các dòng bị lặp không?

In [6]:
duplicated_rows = data.duplicated().sum()
if duplicated_rows > 0:
    print("Số dòng lặp: ", duplicated_rows)
else:
    print("Không có dòng lặp")

Không có dòng lặp


Dữ liệu được thêm vào mỗi ngày nên tạm thời ta không nhận xét gì thêm ở đây

### Mỗi cột có ý nghĩa gì?

|Tên cột| Mô tả |
|:-:| :--|
|**Date**| Ngày lấy số liệu |
|**Country**| Nước |
|**Total Cases**| tổng số ca nhiễm mới từ trước đến nay |
|**New Cases**| số ca mới ngày hôm nay |
|**Total Deaths**| tổng số ca tử vong từ trước đến nay  |
|**New Deaths**| số ca tử vong mới ngày hôm nay|
|**Total Recovered**| tổng số ca phục hồi |
|**New Recovered**| số ca phục hồi trong ngày|
|**Active Cases**| số ca đang điều trị |
|**Serious Critical**| số ca nguy kịch |
|**Total Cases/1M pop**| số ca nhiễm / 1 triệu dân |
|**Deaths/1M pop**| số ca từ vong/ 1 triệu dân |
|**Total Tests**| tổng số xét nghiệm |
|**Tests/1M pop**| số xét nghiệm/1 triệu dân |
|**Population**| dân số |

### Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

Giờ ta xem qua kiểu dữ liệu của các cột nhằm phát hiện cột có kiểu dữ liệu chưa phù hợp

In [7]:
data.dtypes

Date                 object
Country              object
Total Cases           int64
New Cases           float64
Total Deaths        float64
New Deaths          float64
Total Recovered     float64
New Recovered       float64
Active Cases        float64
Serious Critical    float64
Tot Cases/1M pop    float64
Deaths/1M pop       float64
Total Tests         float64
Tests/1M pop        float64
Population          float64
dtype: object

Ta nhận thấy những trừ cột `Data` và `Country` ra thì những cột còn lại nên chuyển về kiểu dữ liệu `int` (đúng về mặt ngữ nghĩa).

Bên cạnh đó, ta cần chuyển cột Date sang kiểu dữ liệu `datetime`

### Sự phân bố giá trị của mỗi cột

#### Với mỗi cột có kiểu dữ liệu dạng numeric, các giá trị được phân bố như thế nào?

Với các cột có kiểu dữ liệu số, ta tính:

- Tỉ lệ % (từ 0 đến 100) các giá trị thiếu
- Giá trị min
- Giá trị lower quartile (phân vị 25)
- Giá trị median (phân vị 50)
- Giá trị upper quartile (phân vị 75)
- Giá trị max

**Nhận xét**:

#### Với mỗi cột có kiểu dữ liệu dạng categorical, các giá trị được phân bố như thế nào?

Với các cột có kiểu dữ liệu dạng `categorical`, ta cần tính:

- Tỉ lệ % (từ 0 đến 100) các giá trị thiếu
- Số lượng các giá trị khác nhau (không xét giá trị thiếu):
- Tỉ lệ % (từ 0 đến 100) của mỗi giá trị được sort theo tỉ lệ % giảm dần (không xét giá trị thiếu)

**Nhận xét**:

### Tiền xử lý dữ liệu